In [1]:
import os
import openai
import sys
from config import set_environment
set_environment
OPENAI_API_KEY="sk-r5WiHoHJBDbj"
os.environ["OPEN_API_KEY"]=OPENAI_API_KEY
openai.api_key=os.environ['OPEN_API_KEY']

# Note  I'm using here Wandb(weights & biases)

if you want you can use Langsmith too

In [2]:
import wandb

In [ ]:
!wandb login 94bd863cc6cf0 #(use your API Key)

In [ ]:
os.environ["WANDB_PROJECT"]="chat_bot"
os.environ["WANDB_LOG_MODEL"]="true"
os.environ["WANDB_WATCH"]="true"

In [3]:
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
persist_directory='docs_1/chroma/'
embedding=OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
vectordb=Chroma(persist_directory=persist_directory,embedding_function=embedding)

C:\Users\Jeevan Reddy\Anaconda\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.embeddings.openai.OpenAIEmbeddings` was deprecated in langchain-community 0.1.0 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import OpenAIEmbeddings`.
  warn_deprecated(


In [4]:
q="what is the Worldwide Smartphone Shipment Forecast"
docs=vectordb.similarity_search(q,k=3)
len(docs)

3

In [5]:
from langchain.chat_models import ChatOpenAI
llm=ChatOpenAI(openai_api_key=os.environ["OPEN_API_KEY"],model_name="gpt-3.5-turbo",temperature=0)
llm.predict("Hello world")

C:\Users\Jeevan Reddy\Anaconda\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The class `langchain_community.chat_models.openai.ChatOpenAI` was deprecated in langchain-community 0.0.10 and will be removed in 0.2.0. An updated version of the class exists in the langchain-openai package and should be used instead. To use it run `pip install -U langchain-openai` and import as `from langchain_openai import ChatOpenAI`.
  warn_deprecated(
C:\Users\Jeevan Reddy\Anaconda\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `predict` was deprecated in LangChain 0.1.7 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'Hello! How can I assist you today?'

In [6]:
from langchain.prompts import PromptTemplate
template="""Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer. Use three sentences maximum. Keep the answer as concise as possible. Always say "thanks for asking!" at the end of the answer. 
{context}
Question:{question}
Helpful Answer:"""
QA_CHAIN_PROMPT=PromptTemplate(input_variables=["context","Question"],template=template)

#run chain
from langchain.chains import RetrievalQA
qa_chain=RetrievalQA.from_chain_type(llm,
                                     retriever=vectordb.as_retriever(),
                                     return_source_documents=True,
                                     chain_type_kwargs={"prompt": QA_CHAIN_PROMPT})
result=qa_chain({"query":q})
result["result"]

C:\Users\Jeevan Reddy\Anaconda\myenv\Lib\site-packages\langchain_core\_api\deprecation.py:117: LangChainDeprecationWarning: The function `__call__` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(


'The worldwide smartphone shipment forecast is expected to reach 1.54 billion units in 2025 with a compound annual growth rate (CAGR) of 3.9% from 2020 to 2025. Thanks for asking!'

In [7]:
from langchain.memory import ConversationBufferMemory
memory=ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

In [8]:
from langchain.chains import ConversationalRetrievalChain
retriever=vectordb.as_retriever()
qa=ConversationalRetrievalChain.from_llm(
    llm,
    retriever=retriever,
    memory=memory)

In [9]:
q1="what are the Secondhand Smartphone Brands Preference"
result=qa({"question":q1})
result["answer"]

'According to the information provided, the brand preferences for secondhand smartphones in India are as follows:\n\n1. Xiaomi\n2. Samsung\n3. Vivo\n4. Realme\n5. OPPO\n6. Apple\n7. OnePlus\n8. Lenovo\n9. Others\n\nPlease note that these preferences are based on the data from IDC and may vary over time.'

In [10]:
q2="what is the top brand"
result=qa({"question":q2})
result["answer"]

'The top preference for secondhand smartphones in India is Xiaomi.'

wandb: Appending key for api.wandb.ai to your netrc file: C:\Users\Jeevan Reddy\.netrc


In [11]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter, RecursiveCharacterTextSplitter
from langchain.vectorstores import DocArrayInMemorySearch
from langchain.document_loaders import TextLoader
from langchain.chains import RetrievalQA,  ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import TextLoader
from langchain.document_loaders import PyPDFLoader